<a href="https://colab.research.google.com/github/aminoside/hello-world/blob/master/Projet2OpenCL_CLEAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


df = pd.read_csv("data.csv", low_memory=False, delimiter='\t')


FileNotFoundError: [Errno 2] No such file or directory: 'data.csv'

In [ ]:
df.shape
df.columns.tolist()

In [ ]:
def supprimer_colonnes_faible_remplissage(dataframe, seuil=60):
    taux_remplissage = dataframe.notnull().mean() * 100
    utiles = taux_remplissage > seuil
    choix = taux_remplissage[utiles]
    return choix.index

In [ ]:
col_utiles=supprimer_colonnes_faible_remplissage(df)
df60 = df[col_utiles]
df60

In [ ]:
df60.info(verbose=True, show_counts=True)

In [ ]:
inutiles=['last_modified_t','last_modified_datetime','creator','created_t','created_datetime','code','url','states_fr',
          'states_tags','states','countries','countries_tags','brands_tags','nutrition-score-uk_100g','serving_size']

In [ ]:
df60['pnns_groups_1']=df['pnns_groups_1'].copy()

In [ ]:
df60=df60.drop(inutiles, axis=1)
dfmean = df60.notnull().mean() * 100
dfmean.sort_values(axis=0, ascending=False, inplace=True)
dfmean

In [ ]:
df60.index
# crée un index des en-têtes de colonnes restantes

In [ ]:
dfmean=pd.DataFrame(data=dfmean.values.reshape(1,-1), columns=dfmean.index)
plt.subplots(figsize=(6,6))
sns.barplot(data=dfmean, orient="h")
plt.xlabel("Taux de remplissage")
# Affiche les taux de remplissage des colonnes de dfmean triés

In [ ]:
df60['energy_100g'].max()

In [ ]:
engie_df = df60[df60['energy_100g'] > 3700]
engie_df

In [ ]:
df60['energy_100g'] = df60['energy_100g'].apply(lambda x: np.nan if x > 3700 else x)

In [ ]:
df60['energy_100g'] = df60['energy_100g'].apply(lambda y: np.nan if y < 0 else y)

In [ ]:
colonnes_float = df60.select_dtypes(include=['float']).columns.tolist()
colonnes_float

In [ ]:
from sklearn.impute import SimpleImputer
imp_const = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0)
df60['fiber_100g'] = imp_const.fit_transform(df60[['fiber_100g']])

In [ ]:
df60[colonnes_float].info()

In [ ]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=3)
array_imputed = imputer.fit_transform(df60[colonnes_float])
df60.loc[:,colonnes_float] = array_imputed

In [ ]:
dfmean = df60.notnull().mean() * 100
dfmean.sort_values(axis=0, ascending=False, inplace=True)
dfmean=pd.DataFrame(data=dfmean.values.reshape(1,-1), columns=dfmean.index)
plt.subplots(figsize=(6,6))
sns.barplot(data=dfmean, orient="h")
plt.xlabel("Taux de remplissage")
# Affiche les taux de remplissage des colonnes de dfmean triés

In [ ]:
df60.info()

In [ ]:
df60.to_csv("donnees_nettoyees.csv", sep=',', index_label=False)